In [1]:
# Jan 30, 2020
# Joined the 5 classification tables available
# to be evaluated:
    # join is left by default
    # because I adopt cit_received as the leftmost data
        # this dataset keep data on the patents who received citation
    # I can make the first join as an 'outer' operation between cit_received and cit_made
    # To be complete, I can begin with patent.csv and make the first two joins with 'outer' type
#     Ill git add now and try this
    

# Jan 29, 2020
# many errors appeared after joining the dataset 
# i rewritten code into simpler scripts
# this script joins variables together to be processed later

# they are:
#     - cit_delay - reads clean_patent and clean_uspatentcitation and calculates delay by patent
#     - cit_made - reads clean_uspatentcitation and calculates citations made 
#     - cit_received - reads clean_uspatentcitation and calculates citations received
#     - cit_tree - reads clean_uspatentcitation and cit_made and calculates parent_citation
#     - generalit - reads wipo and clean_uspatentcitation and calculates generality
#     - originality - reads wipo and clean_uspatentcitation and calculates originality
#     - wipo_first_class - reads wipo and generates wipo_first_class

# it is now way faster, but I cannot know for sure if this is a consequence of coding or something changed in 
# infrastructure. 

# since class is a variable that also appear in non-cited patents, I'll include them in the final step


# Jan, 20th 2020
# join delay and tree data to form the analysis dataset


In [2]:
import pandas as pd
import numpy as np

In [3]:
def data_read(file, names, usecols, dtype):
    df = pd.read_csv(file, names=names, usecols=usecols, dtype=dtype, index_col='id', header=0)
    dfs.append(df)

In [4]:
patent='data/cleanpatent.csv'

received='data/cit_received.csv'
made='data/cit_made.csv'
received_delay='data/cit_received_delay.csv'
made_delay='data/cit_made_delay.csv'
cit_tree = 'data/cit_tree.csv'
originality = 'data/originality.csv'
generality = 'data/generality.csv'

dst='data/dataset.csv'

wipo = 'data/wipo.csv'
ipcr = 'data/ipcr.csv'
cpc = 'data/cpc.csv'
nber = 'data/nber.csv'
uspc = 'data/uspc.csv'

In [5]:
dfs=[]

In [6]:
#adopting patent.csv as the reference of patents
df = pd.read_csv(patent, dtype=object)
df['num_claims']=df['num_claims'].astype(float) #int does not handle NAN values, so using float instead
df.set_index('id', inplace=True)

In [7]:
#outer join to citation received
#there are patents who received citations that are not in patent.csv
#for example, the very ancient ones

names=['index', 'id', 'cit_received']
usecols=['id', 'cit_received']
dtype={'id':object, 'cit_received':float}
df2 = pd.read_csv(received, names=names, usecols=usecols, dtype=dtype, header=0)
df2.set_index('id', inplace=True)

df=df.join(df2, how='outer')
df['cit_received']=df['cit_received'].fillna(0)
dfs.append(df)
df2=[]

In [8]:
#i dont see any reason for patents making citations not to appear in patent.csv
#so from this data on, the list of patents is stable
#the number of rows should not change (check)

names=['index', 'id', 'cit_made']
usecols=['id', 'cit_made']
dtype={'id':object}
data_read(made, names, usecols, dtype)

In [9]:
names=['id', 'cit_received_delay']
dtype={'id':object}
usecols=['id', 'cit_received_delay']
data_read(received_delay, names, usecols, dtype)

In [10]:
names=['id', 'cit_made_delay']
dtype={'id':object}
usecols=['id', 'cit_made_delay']
data_read(made_delay, names, usecols, dtype)

In [11]:
names=['id', 'parent_citation']
dtype={'id':object}
usecols=['id', 'parent_citation']
data_read(cit_tree, names, usecols, dtype)

In [12]:
names=['id', 'df_squared','total_citation','herfindal','originality']
dtype={'id':object}
usecols=['id', 'originality']
data_read(originality, names, usecols, dtype)

In [13]:
names=['id', 'df_squared','total_citation','herfindal','generality']
dtype={'id':object}
usecols=['id', 'generality']
data_read(generality, names, usecols, dtype)

In [14]:
df = pd.read_csv(wipo, dtype={'id':object, 'wipo_field_id':object})
df.set_index('id', inplace=True)
dfs.append(df)

In [15]:
df = pd.read_csv(ipcr, dtype=object)
df.set_index('id', inplace=True)
dfs.append(df)

In [16]:
df = pd.read_csv(cpc, dtype=object)
df.set_index('id', inplace=True)
dfs.append(df)

In [17]:
df = pd.read_csv(nber, dtype=object)
df.set_index('id', inplace=True)
dfs.append(df)

In [18]:
df = pd.read_csv(uspc, dtype=object)
df.set_index('id', inplace=True)
dfs.append(df)

In [19]:
%time
# dfs = [df, df_made, df_received_delay]
# dfs = [df.set_index('id') for df in dfs]
df=dfs[0].join(dfs[1:])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.48 µs


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8702317 entries, 0000000 to re25727
Data columns (total 22 columns):
type                   object
date                   object
kind                   object
num_claims             float64
cit_received           float64
cit_made               float64
cit_received_delay     float64
cit_made_delay         float64
parent_citation        float64
originality            float64
generality             float64
wipo_field_id          object
ipcr_section           object
ipcr_ipc_class         object
ipcr_subclass          object
cpc_section_id         object
cpc_subsection_id      object
cpc_group_id           object
nber_category_id       object
nber_subcategory_id    object
uspc_mainclass_id      object
uspc_subclass_id       object
dtypes: float64(8), object(14)
memory usage: 1.5+ GB


In [21]:
df.sample(n=10)

,type,date,kind,num_claims,cit_received,cit_made,cit_received_delay,cit_made_delay,parent_citation,originality,...,ipcr_section,ipcr_ipc_class,ipcr_subclass,cpc_section_id,cpc_subsection_id,cpc_group_id,nber_category_id,nber_subcategory_id,uspc_mainclass_id,uspc_subclass_id
id,,,,,,,,,,,,,,,,,,,,,
8881968,utility,2014-11-11,B2,16.0,0.0,53.0,NaN,16.793538,NaN,0.346250,...,B,65,D,B,B65,B65D,6,69,229,229/110
20030072134,NaN,NaN,NaN,NaN,1.0,NaN,1.457534,NaN,39.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2860483,NaN,NaN,NaN,NaN,2.0,NaN,46.534247,NaN,17.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9050672,utility,2015-06-09,B2,9.0,0.0,6.0,NaN,30.487215,NaN,0.277778,...,B,23,H,B,B23,B23H,NaN,NaN,No longer published,No longer published
8725473,utility,2014-05-13,B2,14.0,0.0,4.0,NaN,13.037671,NaN,0.000000,...,G,06,F,G,G02,G02C,2,25,703,703/7
4177491,utility,1979-12-04,A,5.0,11.0,6.0,7.775342,4.789498,197.0,0.000000,...,G,11,B,G,G11,G11B,2,24,360,360/135
9745198,utility,2017-08-29,B2,19.0,0.0,1.0,NaN,11.169863,NaN,0.000000,...,C,1,B,C,C01,C01B,NaN,NaN,NaN,NaN
4983609,utility,1991-01-08,A,15.0,4.0,11.0,17.538356,4.447572,15.0,0.165289,...,A,61,K,C,C07,C07D,3,31,514,514/274
1097741,NaN,NaN,NaN,NaN,1.0,NaN,86.254795,NaN,32.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df.describe()

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,num_claims,cit_received,cit_made,cit_received_delay,cit_made_delay,parent_citation,originality,generality
count,6.488254e+06,8.702317e+06,6.183715e+06,7.018034e+06,5.972155e+06,6.256483e+06,5.604693e+06,6.552762e+06
mean,1.414322e+01,1.050814e+01,1.477889e+01,2.036854e+01,1.369109e+01,2.472962e+02,2.742906e-01,2.534382e-01
std,1.200423e+01,2.794500e+01,4.194739e+01,2.422966e+01,1.066944e+01,1.588333e+03,2.662071e-01,2.652160e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,-3.514364e+01,-2.805233e+02,1.000000e+00,0.000000e+00,0.000000e+00
25%,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,3.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,1.000000e+01,NaN,NaN,NaN,NaN,NaN,NaN
max,8.870000e+02,3.922000e+03,5.811000e+03,2.075753e+02,1.827425e+02,2.274790e+05,9.391847e-01,9.235959e-01


In [23]:
df.describe(include=[np.object])

,type,date,kind,wipo_field_id,ipcr_section,ipcr_ipc_class,ipcr_subclass,cpc_section_id,cpc_subsection_id,cpc_group_id,nber_category_id,nber_subcategory_id,uspc_mainclass_id,uspc_subclass_id
count,6488253,6488256,6488253,5879781,5938021,5938021,5938021,5860545,5860545,5860545,5100341,5100341,6288303,6288303
unique,7,2218,15,35,35,147,295,8,122,625,7,38,497,131123
top,utility,2017-04-18,B2,6,G,1,B,G,H04,G06F,2,21,No longer published,No longer published
freq,5879882,7602,2920417,528686,1492884,599269,1061591,1453783,568315,357971,1206890,420382,642999,642999


In [24]:
# df=df.merge(wipo, left_index=True, right_index=True, how='outer')

In [25]:
# df.sample(n=10)

In [26]:
# df.describe()

In [27]:
df.to_csv(dst)

In [28]:
# this code includes WIPO in the dataset
# as class is also part of non-cited patents, I'm not including in here.
# in any case, for some reason this merge could not be done by the method used in the previous
# there is a bug when using read_csv using dtype and indexing. 
# it is a known bug in the community but i do not know why the previous join did not accused the issue
# the method below corrects the issue 

# names=['index', 'id', 'field_id']
# dtype={'id':object}
# usecols=['id', 'field_id']
# # data_read(wipo, names, usecols, dtype)
# wipo = pd.read_csv(wipo, names=names, usecols=usecols, dtype=dtype, header=0)
# wipo['id']=wipo['id'].astype(str)
# wipo=wipo.set_index('id')
# wipo.info()

# dfs.append(wipo)